In [ ]:
#prepare_data.py
# Import necessary modules
#the dataset was taken from kaggle
import os
import numpy as np
import cv2
import kagglehub
from utils import get_face_landmarks
from collections import Counter

In [ ]:
# Download dataset
#The code downloads a dataset using kagglehub, prints the dataset path and its root contents, and initializes an empty list output to store processed data.
path = kagglehub.dataset_download("gauravsharma99/fer13-cleaned-dataset")
# Print the dataset path obtained from KaggleHub
print("Dataset path from KaggleHub:", path)
print("Root contents:", os.listdir(path))
output = []

In [ ]:
# Map class folders to labels (lowercase to handle casing)
#we have 7 emotions from the FER-13 dataset but we are only using first five from the cleaned dataset.
emotion_to_label = {
    'angry': 0,
    'disgust': 1,
    'fear': 2,
    'happy': 3,
    'neutral': 4,
    'sad': 5,
    'surprise': 6
}


In [ ]:
# Loop over folders in the dataset root directory
for emotion_label in os.listdir(path):
    folder_path = os.path.join(path, emotion_label)
    # Skip if the current item is not a directory
    if not os.path.isdir(folder_path):
        continue

    # Loop over image files in the current emotion folder
    for image_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_file)
        # Read the image using OpenCV
        image = cv2.imread(image_path)
        # Skip if the image could not be read
        if image is None:
            continue

        # Resize the image to 192x192 for MediaPipe compatibility
        image = cv2.resize(image, (192, 192))
        # Convert grayscale images to BGR format
        if len(image.shape) == 2 or image.shape[2] == 1:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        # Skip images with unexpected channel dimensions
        elif image.shape[2] != 3:
            print(f"Skipping image with unexpected shape: {image.shape}")
            continue

        # Extract face landmarks using the utility function
        face_landmarks = get_face_landmarks(image)
        # Check if valid landmarks are detected and the count matches the expected size
        if face_landmarks and len(face_landmarks) == 1404:
            # Map the folder name (emotion) to its corresponding label index
            label_index = emotion_to_label.get(emotion_label.lower())
            # If the label index exists, append it to the landmarks and add to the output
            if label_index is not None:
                face_landmarks.append(label_index)
                output.append(face_landmarks)

    # Print a message indicating the folder has been processed
    print(f"✔ Processed emotion folder: {emotion_label}")


In [ ]:
# Report and save
label_counts = Counter([sample[-1] for sample in output])
print("\nSamples per emotion label:")
for label, count in sorted(label_counts.items()):
    name = [k for k, v in emotion_to_label.items() if v == label][0]
    print(f"{label} ({name}): {count}")

np.savetxt("data.txt", np.asarray(output))
print(f"Saved {len(output)} samples to data.txt")

In [ ]:
#now since the data is prepared, we will train the model
#this will give us the accuracy and confusion matrix for each emotion
# Import necessary libraries for data preprocessing, model training, and evaluation
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load and split data
data = np.loadtxt("data.txt")
X, y = data[:, :-1], data[:, -1]

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
# Train model
model = MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=500, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy * 100:.2f}%")
print("\n Classification Report:")
# print(classification_report(y_test, y_pred, target_names=[
#     'Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'
# ]))

# conf_matrix = confusion_matrix(y_test, y_pred)
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
#             xticklabels=['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'],
#             yticklabels=['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'])
unique_labels = sorted(np.unique(y_test).astype(int))
#We are just using the first five emotions from the cleaned dataset.
emotion_map = {
    0: 'Angry',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Neutral',
    5: 'Sad',
    6: 'Surprise'
}
target_names = [emotion_map[i] for i in unique_labels]

In [ ]:
#results: accuracy and confusion matrix
print(classification_report(y_test, y_pred, target_names=target_names))

conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names,
            yticklabels=target_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Emotion Confusion Matrix")
plt.tight_layout()
plt.show()

# Save model and scaler
with open("model", "wb") as f:
    pickle.dump((scaler, model), f)

print("Model and scaler saved as 'model'")


In [ ]:
# get_gpt_tip.py
#this cell is responsible for getting the feedback from gpt based on the emotion detected using system prompting 
import os
import requests
from dotenv import load_dotenv

load_dotenv(".env.local")
api_key = os.getenv("OPENROUTER_API_KEY")

def get_feedback_from_gpt(emotion):
    system_prompt = "You are a professional public speaking coach giving advice based on audience facial emotion. The topic being presented is Hawaiian beach. Give me suggestions based on the audience reachtion specific to the topic. For example, if the expression is Fear, mention an interesting fact about Hawaiian beach that can help the audience feel more comfortable. Give like two or three responses."
    # system_prompt = "You are a professional public speaking coach giving advice based on audience facial emotion. Give like two or three responses."
    user_prompt = f"The audience looks {emotion.lower()}. What should I say or do in my presentation?"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "HTTP-Referer": "https://your-site.com",  # optional
        "X-Title": "Emotion-Based Feedback"
    }

    json_data = {
        "model": "openai/gpt-3.5-turbo",  # or another OpenRouter-supported model
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=json_data)
    data = response.json()
    return data['choices'][0]['message']['content']

# Test
if __name__ == "__main__":
    emotion = input("Enter detected emotion: ")
    tip = get_feedback_from_gpt(emotion)
    print("\nGPT Tip:\n", tip)


In [ ]:
import tkinter as tk
#this cell is reponsible for creating the GUI for the feedback
class EmotionFeedbackGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Live Emotion Feedback")
        self.label = tk.Label(self.root, text="Emotion: None", font=("Arial", 16))
        self.label.pack()
        self.feedback_box = tk.Text(self.root, wrap=tk.WORD, width=50, height=10)
        self.feedback_box.pack()

    def update_feedback(self, emotion, feedback):
        self.label.config(text=f"Emotion: {emotion}")
        self.feedback_box.delete("1.0", tk.END)
        self.feedback_box.insert(tk.END, feedback)

    def run(self):
        self.root.mainloop()


In [ ]:
import cv2
#This script is part of a real-time sentiment analysis project. It utilizes various libraries and modules 
#to perform tasks such as screen capturing, facial emotion analysis, and providing feedback through a GUI.


import mss
import numpy as np
import os
import requests
import textwrap
from deepface import DeepFace
from dotenv import load_dotenv
from emotion_feedback_gui import EmotionFeedbackGUI

In [2]:
# Load GPT API key
load_dotenv(".env.local")
api_key = os.getenv("OPENROUTER_API_KEY")

# Setup GUI
gui = EmotionFeedbackGUI()

NameError: name 'load_dotenv' is not defined

def get_feedback_from_gpt(emotion):
    system_prompt = (
        "You are a professional public speaking coach. "
        "Give a short tip (1-2 sentences) based on the audience's facial emotion. "
        "Be supportive and give suggestions if the audience seems unengaged. "
        f"The topic being presented is Hawaiian beach. If the audience looks {emotion}, tailor advice to that."
    )
    user_prompt = f"The audience looks {emotion.lower()}. What should I say or do next in my presentation?"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "HTTP-Referer": "https://your-site.com",
        "X-Title": "Live Emotion Feedback"
    }

    json_data = {
        "model": "openai/gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    }

    try:
        res = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=json_data)
        res.raise_for_status()
        return res.json()['choices'][0]['message']['content']
    except Exception as e:
        print("GPT error:", e)
        return "Unable to fetch suggestion."

def main():
    with mss.mss() as sct:
        monitor = sct.monitors[1]  # Full screen

        while True:
            screenshot = sct.grab(monitor)
            frame = np.array(screenshot)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

            try:
                result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
                emotion = result[0]['dominant_emotion']
                print(f"\nDetected Emotion: {emotion}")
                suggestion = get_feedback_from_gpt(emotion)
                print(f"GPT Suggestion:\n{textwrap.fill(suggestion, width=80)}\n")
                
                # FIXED: Run GUI update safely in main thread
                gui.root.after(0, gui.update_feedback, emotion, suggestion)

            except Exception as e:
                print(f"DeepFace failed:", e)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    gui.root.after(100, main)  # Schedule emotion loop
    gui.run()
